In [1]:
### Check Meta-Analysis results
### Load the file of co-eQTL mapping from the meta-analysis (all datasets)
### Apply multiple testing to the meta-analysis files and save the significant co-eQTLs and p-value thresholds

# Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1//lib/R/library"



Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: limma

Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.9-1. For overview type 'help("mgcv-package")'.

Loading required package: genefilter

Loading required package: BiocParallel

corrplot 0.95 loaded

Loading required package: viridisLite


Attaching package: ‘purrr’


The following object is masked from ‘package:data.table’:

    transpose



Attaching package: ‘MASS’


The following object is masked from ‘package:genefilter’:

    area


The following object is masked from ‘package:biomaRt’:

    select



Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:data.table’:

    between, first, last


The following object is masked from ‘package:biomaRt’:

    selec

# Parameters

In [5]:
### Cell-type for which to execute

In [6]:
cell_type = 'B'

In [7]:
### Which co-eQTL results (final-mapping: co_qtls_decision_tree; or initial mapping in evaluation phase: co_qtls_sceqtlgen )

In [8]:
coeqtl_results_path = 'co_qtls_decision_tree/'   # path to new results
coeqtl_results_path_old = 'co_qtls_sceqtlgen/'   # path to old results

In [9]:
### Path to the co-EQTL Files

In [10]:
data_path = '../data/current/coeqtl_mapping/'

In [11]:
### Path to the eQTL files (sceQTLGen)

In [12]:
eqtl_results = 'sc_eqtlgen_final'

In [13]:
data_path_eqtl = '../data/current/eqtl_mapping/'

In [14]:
## which meta_analysis (which datasets where included)

In [15]:
meta_analysis_type = 'meta_5ds'

In [16]:
file_name = 'meta_qtl_results_all.txt.gz'

In [17]:
### Parameters for multiple testing correction

In [18]:
n_samples_filter =  133 # either 0 or 133; whether to filter tests that did not include a minimum number of samples

In [19]:
snp_filter = ''

In [20]:
### Path to the significant co-eqtl results

In [21]:
result_path_analysis = paste0( "../data/current/coeqtl_mapping/", coeqtl_results_path , "analysis")

In [22]:
data_path_meta = '../data/current/meta-data'

# Data

## Load ribosomal/mitochondrial genes

In [17]:
### Load the annotation of ribosomal and mitochondrial genes

In [18]:
ribosomal_genes = read.csv(paste0(data_path_meta , '/ribosomal_genes.txt.gz') ,sep = '\t')

In [19]:
ribosomal_genes = unique(ribosomal_genes$Gene.name)

In [20]:
head(ribosomal_genes)

[1] "ABCE1" "ABCF1" "APEX1" "APOD"  "ASCC2" "ASCC3"

In [21]:
### Mitochondrial genes

In [22]:
mitochondrial_genes = read.csv(paste0(data_path_meta , '/mitochondrial_genes.txt') ,sep = '\t')

In [23]:
head(mitochondrial_genes,2)

,Gene.stable.ID,Gene.stable.ID.version,Transcript.stable.ID,Transcript.stable.ID.version,Chromosome.scaffold.name,Gene.start..bp.,Gene.end..bp.
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
1,ENSG00000198888,ENSG00000198888.2,ENST00000361390,ENST00000361390.2,MT,3307,4262
2,ENSG00000198763,ENSG00000198763.3,ENST00000361453,ENST00000361453.3,MT,4470,5511


In [24]:
### convert ENSG to SYMBOL

In [25]:
mitochondrial_genes_convert <- bitr(unique(mitochondrial_genes$Gene.stable.ID), 
                  fromType = "ENSEMBL", 
                  toType = "SYMBOL", 
                  OrgDb = org.Hs.eg.db)

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(unique(mitochondrial_genes$Gene.stable.ID), fromType = "ENSEMBL", :
"2.14% of input gene IDs are fail to map..."


In [26]:
head(mitochondrial_genes_convert,2)

,ENSEMBL,SYMBOL
,<chr>,<chr>
1,ENSG00000198888,ND1
2,ENSG00000198763,ND2


In [27]:
mitochondrial_genes  = unique(mitochondrial_genes_convert$SYMBOL)

In [28]:
head(mitochondrial_genes)

[1] "ND1"  "ND2"  "COX1" "COX2" "ATP8" "ATP6"

In [29]:
### Combine

In [30]:
rb_mt_genes = unique(c(ribosomal_genes, mitochondrial_genes))

## Load the SNP eGene mapping (cis-eQTLs from sceQTLgen)

In [31]:
snp_gene_mapping = read.csv(paste0(data_path_eqtl, eqtl_results , '/' , cell_type, '_snp_egene_mapping.csv'))

In [32]:
nrow(snp_gene_mapping)

[1] 2487

In [33]:
head(snp_gene_mapping,2)

,X,feature_id,snp_id,feature_start,feature_end,p_value,beta
,<int>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
1,1,CHI3L2,1:111230252:A:AT,111200771,111243440,8.658187e-104,1.0929843
2,2,FCRL3,1:157699488:C:T,157674321,157700857,7.538984e-89,-0.6075662


In [34]:
snp_gene_mapping = unique(snp_gene_mapping[,c('feature_id', 'snp_id')])
colnames(snp_gene_mapping) = c('gene', 'snp')

In [35]:
length(unique(snp_gene_mapping$gene))

[1] 2017

## Load Meta-Analysis co-eQTL mapping file

In [42]:
### Load meta-analysis file

In [43]:
cell_type

[1] "B"

In [44]:
meta_analysis = fread(paste0(data_path,coeqtl_results_path , '/meta_analysis/', meta_analysis_type, '/', cell_type, '/', file_name))

In [50]:
path = paste0(data_path,coeqtl_results_path , '/meta_analysis/', meta_analysis_type, '/', cell_type, '/', file_name)

In [51]:
file.info(path)$ctime

[1] "2025-03-03 11:54:43 CET"

In [57]:
meta_analysis[meta_analysis$feature_id ==  'IGHG2_VCPKMT',1:31]

coQTL,eGene,feature_chromosome,feature_start,feature_end,coeGene,feature_id,snp_id,snp_position,closeGenes,⋯,z_score;oneK1K,n_samples;Franke_V3,n_e_samples;Franke_V3,z_score;Franke_V3,n_samples;Franke_V2,n_e_samples;Franke_V2,z_score;Franke_V2,n_samples;Wijst,n_e_samples;Wijst,z_score;Wijst
<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<lgl>,⋯,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>
14:50315331:G:A;VCPKMT_IGHG2,VCPKMT,14,50108632,50116600,IGHG2,IGHG2_VCPKMT,14:50315331:G:A,50315331,FALSE,⋯,-5.74351200,NA,NA,NA,NA,NA,NA,NA,NA,NA
14:50219111:C:CA;VCPKMT_IGHG2,VCPKMT,14,50108632,50116600,IGHG2,IGHG2_VCPKMT,14:50219111:C:CA,50219111,FALSE,⋯,-1.83053200,NA,NA,NA,NA,NA,NA,NA,NA,NA
14:104896820:A:G;IGHG2_VCPKMT,IGHG2,14,105639559,105644790,VCPKMT,IGHG2_VCPKMT,14:104896820:A:G,104896820,FALSE,⋯,-1.39619300,NA,NA,NA,NA,NA,NA,NA,NA,NA
14:49843683:T:C;VCPKMT_IGHG2,VCPKMT,14,50108632,50116600,IGHG2,IGHG2_VCPKMT,14:49843683:T:C,49843683,FALSE,⋯,0.55119880,NA,NA,NA,NA,NA,NA,NA,NA,NA
14:104924668:C:T;IGHG2_VCPKMT,IGHG2,14,105639559,105644790,VCPKMT,IGHG2_VCPKMT,14:104924668:C:T,104924668,FALSE,⋯,0.30941640,NA,NA,NA,NA,NA,NA,NA,NA,NA
14:105057326:G:A;IGHG2_VCPKMT,IGHG2,14,105639559,105644790,VCPKMT,IGHG2_VCPKMT,14:105057326:G:A,105057326,FALSE,⋯,-0.26904990,NA,NA,NA,NA,NA,NA,NA,NA,NA
14:104919812:G:C;IGHG2_VCPKMT,IGHG2,14,105639559,105644790,VCPKMT,IGHG2_VCPKMT,14:104919812:G:C,104919812,FALSE,⋯,-0.05839607,NA,NA,NA,NA,NA,NA,NA,NA,NA
14:104921845:A:C;IGHG2_VCPKMT,IGHG2,14,105639559,105644790,VCPKMT,IGHG2_VCPKMT,14:104921845:A:C,104921845,FALSE,⋯,-0.04793983,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [59]:
### Only for Mono/ B: filter additionally on eGenes within 5% FDR threshold
meta_analysis = meta_analysis[meta_analysis$eGene %in% unique(snp_gene_mapping$gene),]

In [40]:
nrow(meta_analysis)

[1] 120536750

In [44]:
### Define triplets

In [45]:
meta_analysis$triplet = paste0( meta_analysis$snp_id,  ';', meta_analysis$feature_id)

### Define ribosomal and mitochondrial genes

In [43]:
### Define rb/mt genes
meta_analysis$is_rb_mt = 0
meta_analysis$is_rb_mt[(meta_analysis$eGene %in% rb_mt_genes) |
                            (meta_analysis$coeGene %in% rb_mt_genes)] = 1

In [46]:
### Save the info about the amount of tests per diff gene-type and amount samples

In [47]:
#head(meta_analysis,2)

In [48]:
sort(colnames(meta_analysis))

[1] "closeGenes"            "coeGene"               "coQTL"                
 [4] "eGene"                 "feature_chromosome"    "feature_end"          
 [7] "feature_id"            "feature_start"         "I_square"             
[10] "is_rb_mt"              "n_e_samples"           "n_e_samples;Franke_V2"
[13] "n_e_samples;Franke_V3" "n_e_samples;Multiome"  "n_e_samples;oneK1K"   
[16] "n_e_samples;Wijst"     "n_samples"             "n_samples;Franke_V2"  
[19] "n_samples;Franke_V3"   "n_samples;Multiome"    "n_samples;oneK1K"     
[22] "n_samples;Wijst"       "p_value"               "snp_id"               
[25] "snp_position"          "triplet"               "z_score"              
[28] "z_score_dataset_dev"   "z_score;Franke_V2"     "z_score;Franke_V3"    
[31] "z_score;Multiome"      "z_score;oneK1K"        "z_score;Wijst"

In [49]:
amount_tests_save =meta_analysis %>% group_by( n_samples, is_rb_mt) %>% summarise(amount_tests_datasets = length(unique(triplet)))

`summarise()` has grouped output by 'n_samples'. You can override using the
`.groups` argument.


In [50]:
head(amount_tests_save,2)

n_samples,is_rb_mt,amount_tests_datasets
<int>,<dbl>,<int>
16,0,22
16,1,9


In [51]:
nrow(amount_tests_save)

[1] 2048

In [52]:
write.csv(amount_tests_save, paste0(data_path,coeqtl_results_path , '/meta_analysis/', meta_analysis_type, '/', cell_type, '/','Meta_Info_Tests.csv'))

### Filter on only the cis-eQTL snps

In [54]:
nrow(meta_analysis)

[1] 120536750

In [55]:
if(snp_filter == 'cis'){
    meta_analysis = meta_analysis[meta_analysis$snp_id %in% unique(snp_gene_mapping$snp),]
    }
    

In [56]:
nrow(meta_analysis)

[1] 120536750

In [57]:
nrow(meta_analysis)

[1] 120536750

In [58]:
## Save amount of tests after filtering

In [59]:
amount_tests_save =meta_analysis %>% group_by( n_samples, is_rb_mt) %>% summarise(amount_tests_datasets = length(unique(triplet)))

`summarise()` has grouped output by 'n_samples'. You can override using the
`.groups` argument.


In [60]:
head(amount_tests_save,2)

n_samples,is_rb_mt,amount_tests_datasets
<int>,<dbl>,<int>
16,0,22
16,1,9


In [61]:
nrow(amount_tests_save)

[1] 2048

In [62]:
snp_filter

[1] ""

In [63]:
write.csv(amount_tests_save, paste0(data_path,coeqtl_results_path , '/meta_analysis/', meta_analysis_type, '/', cell_type, '/','Meta_Info_Tests_', snp_filter, '.csv'))

### Filter on minimum number of samples

In [65]:
if (n_samples_filter > 0){
    meta_analysis = meta_analysis[meta_analysis$n_samples > n_samples_filter,]
    }
    

In [66]:
nrow(meta_analysis)

[1] 67453627

In [67]:
length(unique(meta_analysis$feature_id))

[1] 7908909

In [68]:
length(unique(meta_analysis$feature_id))

[1] 7908909

In [69]:
length(unique(meta_analysis$triplet))

[1] 67422435

In [70]:
length(unique(meta_analysis$triplet))

[1] 67422435

In [71]:
amount_tests = length(unique(meta_analysis$triplet))   # get the unique amount of tests for multiple testing correction

In [72]:
class(meta_analysis)

[1] "data.table" "data.frame"

In [73]:
meta_analysis = setDT(meta_analysis)

In [74]:
class(meta_analysis)

[1] "data.table" "data.frame"

### Check for duplicates

In [76]:
dupl_invest =  meta_analysis[, .(.N), by = .(triplet)]

In [77]:
head(dupl_invest,2)

triplet,N
<chr>,<int>
12:56007301:G:A;RPL13_RPS26,1
12:56007301:G:A;RPL10_RPS26,1


In [78]:
head(dupl_invest %>% arrange(-N),2)

triplet,N
<chr>,<int>
6:32658626:A:G;HLA-DQA1_HLA-DRB1,2
6:32632280:A:G;HLA-DQA1_HLA-DRB1,2


In [79]:
nrow(dupl_invest[dupl_invest$N == 2,])

[1] 31192

In [80]:
nrow(dupl_invest[dupl_invest$N == 2,])

[1] 31192

In [81]:
dupl_invest =dupl_invest[dupl_invest$N == 2,]

In [82]:
duplicates = meta_analysis[meta_analysis$triplet %in% dupl_invest$triplet,]

In [83]:
nrow(duplicates)

[1] 62384

In [84]:
unique(duplicates$closeGenes)

[1] TRUE

### Multiple testing correctin and get significant results

In [3]:
### Add p-value with Bonferroni correction

In [88]:
amount_tests

[1] 67422435

In [89]:
amount_tests

[1] 67422435

In [90]:
meta_analysis$p_val_mt = as.numeric(meta_analysis$p_value) * amount_tests
meta_analysis$q_value = qvalue(p = meta_analysis$p_value)$qvalues

In [91]:
nrow(meta_analysis)

[1] 67453627

In [4]:
### Step-wise multiple testing correction

In [93]:
### Get the amount of tests per eGene
meta_analysis = meta_analysis %>% group_by(eGene) %>% mutate(amount_tests_eGene = n())

In [94]:
head(meta_analysis,2)

coQTL,eGene,feature_chromosome,feature_start,feature_end,coeGene,feature_id,snp_id,snp_position,closeGenes,⋯,n_e_samples;Franke_V2,z_score;Franke_V2,n_samples;Wijst,n_e_samples;Wijst,z_score;Wijst,is_rb_mt,triplet,p_val_mt,q_value,amount_tests_eGene
<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<lgl>,⋯,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<int>
12:56007301:G:A;RPS26_RPL13,RPS26,12,56041351,56044676,RPL13,RPL13_RPS26,12:56007301:G:A,56007301,FALSE,⋯,91,9.331416,40,40,5.272181,1,12:56007301:G:A;RPL13_RPS26,2.030599e-130,1.983404e-130,143330
12:56007301:G:A;RPS26_RPL10,RPS26,12,56041351,56044676,RPL10,RPL10_RPS26,12:56007301:G:A,56007301,FALSE,⋯,92,9.062418,40,40,3.646652,1,12:56007301:G:A;RPL10_RPS26,1.597442e-118,7.801572e-119,143330


In [95]:
### Correct for multiple testing per eGene
meta_analysis$p_val_mt_eGene = meta_analysis$p_value * meta_analysis$amount_tests_eGene

In [96]:
min(meta_analysis$p_value)
max(meta_analysis$p_value)

[1] 3.011756e-138

[1] 1

In [97]:
length(unique(meta_analysis$eGene))   # amount of eGenes

[1] 1417

In [98]:
### Get the top effect per eGene
top_eGene_effects = meta_analysis %>% group_by(eGene) %>% dplyr::filter(p_value == min(p_value)) %>% ungroup()

In [99]:
top_eGene_effects = top_eGene_effects %>% group_by(eGene) %>% slice_min(p_value, with_ties = FALSE)

In [100]:
nrow(top_eGene_effects)

[1] 1417

In [104]:
## Set values greater 1 to 1

In [105]:
top_eGene_effects$p_val_mt_eGene[top_eGene_effects$p_val_mt_eGene > 1] = 1

In [101]:
### Correct top-effects with q_value

In [107]:
top_eGene_effects$q_val = qvalue(p = top_eGene_effects$p_val_mt_eGene)$qvalues

#top_eGene_effects$p_bh = qvalue(p = top_eGene_effects$p_val_mt_eGene,pi0 = 1)$qvalues

top_eGene_effects$p_bh =  p.adjust(top_eGene_effects$p_val_mt_eGene, method = "BH")

In [109]:
head(top_eGene_effects,2)

coQTL,eGene,feature_chromosome,feature_start,feature_end,coeGene,feature_id,snp_id,snp_position,closeGenes,⋯,n_e_samples;Wijst,z_score;Wijst,is_rb_mt,triplet,p_val_mt,q_value,amount_tests_eGene,p_val_mt_eGene,q_val,p_bh
<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<lgl>,⋯,<int>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
19:58358721:TG:T;A1BG_POLE3,A1BG,19,58345178,58353499,POLE3,A1BG_POLE3,19:58358721:TG:T,58358721,FALSE,⋯,NA,NA,0,19:58358721:TG:T;A1BG_POLE3,1635.152,0.07942455,19688,0.4774802,0.8775479,0.8775479
4:56348606:C:T;AASDH_PTEN,AASDH,4,56338287,56387508,PTEN,AASDH_PTEN,4:56348606:C:T,56348606,FALSE,⋯,NA,NA,0,4:56348606:C:T;AASDH_PTEN,15414.861,0.26287243,3591,0.8210140,1.0000000,1.0000000


In [24]:
p_val_thres = 0.05

In [111]:
### Get the cutoff p-value
p_val_cutoff_bh = head(top_eGene_effects[top_eGene_effects$p_bh < p_val_thres,c('p_val_mt_eGene', 'p_bh')] %>% arrange(-p_bh),1)
p_val_cutoff_q = head(top_eGene_effects[top_eGene_effects$q_val < p_val_thres,c('p_val_mt_eGene', 'q_val')] %>% arrange(-q_val),1)

In [112]:
p_val_cutoff_bh

p_val_mt_eGene,p_bh
<dbl>,<dbl>
0.002149966,0.04406373


In [113]:
p_val_cutoff_q

p_val_mt_eGene,q_val
<dbl>,<dbl>
0.002149966,0.04406373


In [114]:
## Add the cutoff p-value to the result
meta_analysis$p_bh_cutoff = p_val_cutoff_bh$p_val_mt_eGene
meta_analysis$p_q_cutoff = p_val_cutoff_q$p_val_mt_eGene

In [115]:
nrow(meta_analysis)

[1] 67453627

In [116]:
### Get nominal significant results

In [117]:
sign_meta = meta_analysis[meta_analysis$p_value < 0.05,]
#sign_meta_nom =  meta_analysis[meta_analysis$p_value < 0.05,]

In [119]:
nrow(sign_meta)

[1] 3906619

In [128]:
### Summarise and mark the duplicates

In [129]:
if(nrow(dupl_invest)  > 0){
    ## Previous solution
    # coeqtl_results = coeqtl_results %>% group_by(feature_id, snp_id,   cell_type, dataset, feature_chromosome, feature_end, feature_start) %>% summarise(beta_se = max(beta_se), p_value = mean(p_value),beta = mean(beta)
    
    sign_meta_un = sign_meta[!sign_meta$triplet %in% dupl_invest$triplet,]
    sign_meta_un$type = 'unique'
    
    sign_meta_dupl = sign_meta[sign_meta$triplet %in% dupl_invest$triplet,]
    sign_meta_dupl$eGene = sign_meta_dupl$feature_id
    sign_meta_dupl$coeGene = sign_meta_dupl$feature_id
    sign_meta_dupl$type = 'duplicate'
    
    sign_meta_dupl=    sign_meta_dupl %>% group_by(triplet) %>% mutate(rank = rank(eGene, ties.method = 'first'))
    sign_meta_dupl =  sign_meta_dupl[ sign_meta_dupl$rank == 1,]
    sign_meta_dupl$rank = NULL
    
    sign_meta = rbind(sign_meta_un, sign_meta_dupl)
    
    
    }

In [130]:
length(unique(sign_meta$coeGene))

[1] 7589

In [131]:
length(unique(sign_meta$eGene))

[1] 1955

In [132]:
length(unique(sign_meta$feature_id))

[1] 1514078

In [133]:
nrow(sign_meta)

[1] 3903170

In [134]:
max(sign_meta$p_value)

[1] 0.05

In [ ]:
### Save the nominally significant results

In [135]:
write.csv(sign_meta, paste0(data_path,coeqtl_results_path , '/meta_analysis/', meta_analysis_type, '/', cell_type, '/','Meta_Analysis_Sign_results', n_samples_filter ,'_', snp_filter,p_val_thres,  '.csv'))
#write.csv(sign_meta_nom, paste0(data_path,coeqtl_results_path , '/meta_analysis/', meta_analysis_type, '/', cell_type, '/','Meta_Analysis_Sign_results_nom', n_samples_filter ,'.csv'))

In [136]:
### Generate info about snps giving a co-eqtl and not

In [137]:
meta_analysis$is_coeqtl = 0
meta_analysis$is_coeqtl[meta_analysis$p_val_mt < 0.05] = 1

In [138]:
snp_coeqtl = meta_analysis %>% group_by(snp_id) %>% summarise(is_coeqtl = max(is_coeqtl))

In [139]:
write.csv(snp_coeqtl, paste0(data_path,coeqtl_results_path , '/meta_analysis/', meta_analysis_type, '/', cell_type, '/', 'SNP_Class_', cell_type, '_', meta_analysis_type, '_', snp_filter,p_val_thres, '.csv'))